In [1]:
import pandas as pd
import numpy as np
import sqlite3
from sqlite3 import Error

In [2]:
"""
Add all
"""
add_all = True
save = True

if add_all:
    
    #Load result directory/file
    #result_dir = "/groups/icecube/cjb924/workspace/work/results_from_code_use_models/neutrino_multiclassfication_peter_continued/no_5/pid_test_frederik_on_peter_Burnsample_RD_Full_db_new_muon_no_5.csv"
    #result = pd.read_csv(result_dir).sort_values('event_no').reset_index(drop=True)
    result_dir = "/groups/icecube/cjb924/workspace/work/high_energy/high_energy_models/predicting/predictions/3_model_with_HE/pid_3_model_with_HE_prediction_burnsample.csv"
    result = pd.read_csv(result_dir).sort_values('event_no').reset_index(drop=True)

    #Andreash
    #result_dir = "/groups/icecube/andreash/workspace/training/outputs/train_model_medium_128_randomcropped_he/MuonGunMerged/dynedge_pid/results_real_data.parquet"
    #result_dir = "/lustre/hpc/icecube/andreash/workspace/training/outputs/train_model_medium_randomcropped/MuonGunMerged/dynedge_pid/results_real_data.parquet"
    #result = pd.read_parquet(result_dir).sort_values('event_no').reset_index(drop=True)
    

    #Total osc_weight
    #Path to run time csv_file for the used i3-files
    runtimes = pd.read_csv("/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_0.01_percent_burnsample_Peter/Run_times_to_get_rate/Burnsample_SubRun_livetimes.csv")
    #Putting in the total_osc_weight
    result['total_osc_weight'] = 1/runtimes['livetime_s'].sum()


    #Logit p preds on neutrino
    logit_computations = 'pid_neutrino_pred'
    result_i = result[logit_computations]
    eps = 10**(-7)

    p_eps = result_i * (1 - 2 * eps) + eps
    #p_eps = np.clip(p_eps, eps, 1 - eps)
    log_p = np.log(p_eps / (1 - p_eps))

    log_p_neutrinos = log_p.tolist()

    result['pid_neutrino_pred_logit'] = log_p_neutrinos


    #lvl cleaning passed
    #Load in necessary information (I have already created csv_files as it is faster to load than databases)
    data_cols = ['event_no', 'EventID', 'SubEventID', 'RunID', 'SubrunID']
    info_cols = ['EventID', 'SubEventID', 'RunID', 'SubrunID', 'L3_oscNext_bool']#, 'L4_oscNext_bool', 'L5_oscNext_bool', 'L6_oscNext_bool', 'L7_oscNext_bool']
    df_data = pd.read_csv("/groups/icecube/cjb924/workspace/work/code_work_on_results/df_data.csv", usecols=data_cols)
    df_info = pd.read_csv("/groups/icecube/cjb924/workspace/work/code_work_on_results/df_info.csv", usecols=info_cols)
    #df_info = pd.read_csv('/groups/icecube/cjb924/workspace/work/peter_continued/code_work_on_results/df_info_all_cleaning_cuts.csv', usecols=info_cols)
   
    # Merge the DataFrames based on the 'EventID' and 'RunID' columns 
    merged_df = df_data.merge(df_info, on=['EventID', 'RunID'], how='inner')

    #Putting in the L3_oscNext_bool
    result['L3_oscNext_bool'] = merged_df['L3_oscNext_bool']
    #result['L4_oscNext_bool'] = merged_df['L4_oscNext_bool']
    #result['L5_oscNext_bool'] = merged_df['L5_oscNext_bool']
    #result['L6_oscNext_bool'] = merged_df['L6_oscNext_bool']
    #result['L7_oscNext_bool'] = merged_df['L7_oscNext_bool']

    if save:
        #Output folder
        #output_folder = "/groups/icecube/cjb924/workspace/work/results_from_code_use_models/neutrino_multiclassfication_peter_continued/no_5"
        output_folder = "/groups/icecube/cjb924/workspace/work/high_energy/high_energy_models/predicting/predictions/3_model_with_HE"
        result.to_csv(
            #Give name here
            #output_folder + "/neutrino_multiclassificatio_with_all_cleaning_cuts_no_5.csv"
            output_folder + "/pid_3_model_with_HE_prediction_burnsample_with_all.csv"

        )

    
    



In [6]:
result

,Unnamed: 0,pid_noise_pred,pid_muon_pred,pid_neutrino_pred,pid,event_no,total_osc_weight,pid_neutrino_pred_logit,L3_oscNext_bool,L4_oscNext_bool,L5_oscNext_bool,L6_oscNext_bool,L7_oscNext_bool
0,0,9.004681e-01,0.014497,0.085035,-1.0,0.0,2.553499e-07,-2.375819,0.0,-1.0,-1.0,-1.0,-1.0
1,1,7.181174e-11,0.999545,0.000455,-1.0,1.0,2.553499e-07,-7.694227,0.0,-1.0,-1.0,-1.0,-1.0
2,2,2.273945e-10,0.998961,0.001039,-1.0,2.0,2.553499e-07,-6.868046,0.0,-1.0,-1.0,-1.0,-1.0
3,3,9.305553e-01,0.058857,0.010588,-1.0,3.0,2.553499e-07,-4.537383,0.0,-1.0,-1.0,-1.0,-1.0
4,4,9.270968e-01,0.049770,0.023133,-1.0,4.0,2.553499e-07,-3.743071,0.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61739857,61739857,9.406110e-01,0.050188,0.009201,-1.0,99904418.0,2.553499e-07,-4.679186,0.0,-1.0,-1.0,-1.0,-1.0
61739858,61739858,5.804310e-06,0.684888,0.315106,-1.0,99904419.0,2.553499e-07,-0.776353,1.0,-1.0,-1.0,-1.0,-1.0
61739859,61739859,2.248411e-10,0.997775,0.002225,-1.0,99904420.0,2.553499e-07,-6.105749,0.0,-1.0,-1.0,-1.0,-1.0
61739860,61739860,4.207002e-18,0.931530,0.068470,-1.0,99904421.0,2.553499e-07,-2.610428,0.0,-1.0,-1.0,-1.0,-1.0


In [2]:
"""
Total_osc_weight:
Works kinda right, right now it takes from everything but want to split it up more into i3-files.
"""
total_osc_weight = True
save_total_osc_weight = True

if total_osc_weight:
    #Path to run time csv_file for the used i3-files
    runtimes = pd.read_csv("/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_0.01_percent_burnsample_Peter/Run_times_to_get_rate/Burnsample_SubRun_livetimes.csv")

    #Computing the total_osc_weight
    print('total_osc_weight/inverse sum of all runtime:', runtimes['livetime_s'].sum())

    #Result directory (path) for predicted reults
    result_dir = "/groups/icecube/cjb924/workspace/work/high_energy/high_energy_models/predicting/predictions/2_model_HE/signal_2model_prediction_burnsample_with_extra.csv"
    #result_dir = "/groups/icecube/cjb924/workspace/work/results_from_code_use_models/pid_frederik_on_peter_Burnsample_RD_Full_db_new_muon.csv"
    result = pd.read_csv(result_dir).sort_values('event_no').reset_index(drop=True)
    result

    #Putting in the total_osc_weight
    result['total_osc_weight'] = 1/runtimes['livetime_s'].sum()

    if save_total_osc_weight:
        #Output folder
        output_folder = "/groups/icecube/cjb924/workspace/work/high_energy/high_energy_models/predicting/predictions/2_model_HE"
        result.to_csv(
            #Give name here
            output_folder + "/testing.csv"
        )



total_osc_weight/inverse sum of all runtime: 3916195.75


In [3]:
"""
Computing logit_p_preds from the pred (only on neutrinos).
It works and is quite fast.
"""
make_logit_p_neutrino = True
save_logit_p_neutrino = True

if make_logit_p_neutrino:

    result_dir = "/groups/icecube/cjb924/workspace/work/high_energy/high_energy_models/training/saved_results/2_model_HE/_test_set/results.csv"
    result = pd.read_csv(result_dir).sort_values('event_no').reset_index(drop=True)
    result
    
    logit_computations = 'signal_neutrino_HE_pred'
    result_i = result[logit_computations]
    eps = 10**(-7)
    
    p_eps = result_i * (1 - 2 * eps) + eps
    #p_eps = np.clip(p_eps, eps, 1 - eps)
    log_p = np.log(p_eps / (1 - p_eps))
    
    log_p_neutrinos = log_p.tolist()
    
    result['signal_neutrino_HE_pred_logit'] = log_p_neutrinos

    if save_logit_p_neutrino:
        #Output folder
        output_folder = "/groups/icecube/cjb924/workspace/work/high_energy/high_energy_models/training/saved_results/2_model_HE/_test_set"
        result.to_csv(
            #Give name here
            output_folder + "/2_model_MC_logit_p.csv"
        )


In [4]:
#result_dir = "/groups/icecube/cjb924/workspace/work/results_from_code_use_models/testing.csv"
#result = pd.read_csv(result_dir).sort_values('event_no').reset_index(drop=True)

In [5]:
"""
Adding the L3_oscNext_bool meaning if it survives the lvl 3? cut or not.
"""
L_oscNextbool = False
save_L_oscNextbool = False

if L_oscNextbool:

    #Load in necessary information (I have already created csv_files as it is faster to load than databases)
    data_cols = ['event_no', 'EventID', 'SubEventID', 'RunID', 'SubrunID']
    info_cols = ['EventID', 'SubEventID', 'RunID', 'SubrunID', 'L3_oscNext_bool']
    df_data = pd.read_csv("/groups/icecube/cjb924/workspace/work/peter_continued/code_work_on_results/df_data.csv", usecols=data_cols)
    df_info = pd.read_csv("/groups/icecube/cjb924/workspace/work/peter_continued/code_work_on_results/df_info.csv", usecols=info_cols)
    
    # Merge the DataFrames based on the 'EventID' and 'RunID' columns 
    merged_df = df_data.merge(df_info, on=['EventID', 'RunID'], how='inner')
    
    #Result directory (path) for predicted reults
    result_dir = "/groups/icecube/cjb924/workspace/work/results_from_code_use_models/testing.csv"
    #result_dir = "/groups/icecube/cjb924/workspace/work/results_from_code_use_models/pid_frederik_on_peter_Burnsample_RD_Full_db_new_muon.csv"
    result = pd.read_csv(result_dir).sort_values('event_no').reset_index(drop=True)
    
    #Putting in the L3_oscNext_bool
    result['L3_oscNext_bool'] = merged_df['L3_oscNext_bool']

if save_L_oscNextbool:
        #Output folder
        output_folder = "/groups/icecube/cjb924/workspace/work/results_from_code_use_models"
        result.to_csv(
            #Give name here
            output_folder + "/testing.csv"
        )

'\nAdding the L3_oscNext_bool meaning if it survives the lvl 3? cut or not.\nNot working yey looking into a solution where using the RunID and EventID can be used to merge L3_oscNext_bool into the database as even_no is not unique but they are.\n'

In [4]:
cols = ['EventID', 'SubEventID', 'RunID', 'SubrunID', 'L3_oscNext_bool', 'L4_oscNext_bool', 'L5_oscNext_bool', 'L6_oscNext_bool', 'L7_oscNext_bool']
result = pd.read_csv('/groups/icecube/cjb924/workspace/work/peter_continued/code_work_on_results/df_info_all_cleaning_cuts.csv', usecols=cols)
result

,EventID,SubEventID,RunID,SubrunID,L3_oscNext_bool,L4_oscNext_bool,L5_oscNext_bool,L6_oscNext_bool,L7_oscNext_bool
0,72198962.0,0.0,135223.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0
1,47.0,0.0,119430.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0
2,108.0,0.0,119430.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0
3,72199567.0,0.0,135223.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0
4,72199799.0,0.0,135223.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...
64039514,275680.0,0.0,130415.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0
64039515,275750.0,0.0,130415.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0
64039516,276006.0,0.0,130415.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0
64039517,276228.0,0.0,130415.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0


In [3]:
connection = sqlite3.connect('/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_0.01_percent_burnsample_Peter/merged_db/Burnsample_lvl3_v02.00_lvl3_variables.db')

columns_select = ['EventID', 'SubEventID', 'RunID', 'SubrunID', 'L3_oscNext_bool', 'L4_oscNext_bool', 'L5_oscNext_bool', 'L6_oscNext_bool', 'L7_oscNext_bool']
query = f"SELECT {', '.join(columns_select)} FROM truth"
df_info = pd.read_sql_query(query, connection)

connection.close()

#connection_2 = sqlite3.connect('/groups/icecube/petersen/GraphNetDatabaseRepository/dev_lvl3_genie_burnsample/dev_lvl3_genie_burnsample_v5.db')
#
#columns_select = ['event_no', 'EventID', 'SubEventID', 'RunID', 'SubrunID']#, 'L3_oscNext_bool']
#
#query = f"SELECT {', '.join(columns_select)} FROM truth"
#df_data = pd.read_sql_query(query, connection_2)
#
#connection_2.close()

#Output folder
output_folder = "/groups/icecube/cjb924/workspace/work/peter_continued/code_work_on_results"
df_info.to_csv(
    #Give name here
    output_folder + "/df_info_all_cleaning_cuts.csv"
)
#df_data.to_csv(
#    #Give name here
#    output_folder + "/df_data.csv"
#)

#data_cols = ['event_no', 'EventID', 'SubEventID', 'RunID', 'SubrunID']
#info_cols = ['EventID', 'SubEventID', 'RunID', 'SubrunID', 'L3_oscNext_bool']
#df_data = pd.read_csv("/groups/icecube/cjb924/workspace/work/peter_continued/code_work_on_results/df_data.csv", usecols=data_cols)
#df_info = pd.read_csv("/groups/icecube/cjb924/workspace/work/peter_continued/code_work_on_results/df_info.csv", usecols=info_cols)

## Merge the DataFrames and add an indicator column
#merged_df = df_data[['EventID', 'RunID']].merge(df_info[['EventID', 'RunID']], indicator=True, how='outer')
#
## Filter rows where both DataFrames are equal
#equal_rows = merged_df[merged_df['_merge'] == 'both']
#
## Get the indices from DataFrame 1
#indices_in_df1 = equal_rows.index.tolist()
#
#
#
## Merge the DataFrames and add an indicator column
#merged_df_2 = df_info[['EventID', 'RunID']].merge(df_data[['EventID', 'RunID']], indicator=True, how='outer')
#
## Filter rows where both DataFrames are equal
#equal_rows_2 = merged_df_2[merged_df_2['_merge'] == 'both']
#
## Get the indices from DataFrame 2
#indices_in_df2 = equal_rows_2.index.tolist()

In [5]:
"""
Computing logit_p_preds from the pred (he and le neutrino combined).
It works and is quite fast.
"""
make_logit_p_neutrino = True
save_logit_p_neutrino = True

if make_logit_p_neutrino:

    result_dir = "/groups/icecube/cjb924/workspace/work/high_energy/5_model/training/saved_results/no1_first/_validation_set/results.csv"
    result = pd.read_csv(result_dir).sort_values('event_no').reset_index(drop=True)
    result
    
    
    logit_computations_LE_nu = 'signal_neutrino_LE_pred'
    logit_computations_HE_nu = 'signal_neutrino_HE_pred'
    result_LE = result[logit_computations_LE_nu]
    result_HE = result[logit_computations_HE_nu]
    result_i = result_LE + result_HE
    
    #result_i = result[logit_computations]
    eps = 10**(-7)
    
    p_eps = result_i * (1 - 2 * eps) + eps
    #p_eps = np.clip(p_eps, eps, 1 - eps)
    log_p = np.log(p_eps / (1 - p_eps))
    
    log_p_neutrinos = log_p.tolist()
    
    result['signal_neutrino_HE_plus_LE_pred_logit'] = log_p_neutrinos

    if save_logit_p_neutrino:
        #Output folder
        output_folder = "/groups/icecube/cjb924/workspace/work/high_energy/5_model/training/saved_results/no1_first/_validation_set"
        result.to_csv(
            #Give name here
            output_folder + "/results.csv"
        )


In [3]:
result

,Unnamed: 0,signal_noise_pred,signal_neutrino_LE_pred,signal_neutrino_HE_pred,signal_muon_LE_pred,signal_muon_HE_pred,signal,event_no,signal_neutrino_HE_plus_LE_pred_logit
0,600000,0.000000e+00,0.0,0.000000e+00,1.0,0.000000e+00,3.0,1.0,-16.118096
1,600001,0.000000e+00,0.0,0.000000e+00,1.0,0.000000e+00,3.0,6.0,-16.118096
2,600002,0.000000e+00,0.0,0.000000e+00,1.0,0.000000e+00,3.0,8.0,-16.118096
3,600003,0.000000e+00,0.0,0.000000e+00,1.0,0.000000e+00,3.0,12.0,-16.118096
4,600004,0.000000e+00,0.0,0.000000e+00,1.0,0.000000e+00,3.0,17.0,-16.118096
...,...,...,...,...,...,...,...,...,...
943942,743942,0.000000e+00,0.0,0.000000e+00,1.0,0.000000e+00,3.0,65909110.0,-16.118096
943943,743943,1.008234e-41,0.0,2.541396e-36,1.0,0.000000e+00,3.0,65909130.0,-16.118096
943944,743944,2.898764e-39,0.0,9.980610e-37,1.0,1.401298e-45,3.0,65909147.0,-16.118096
943945,743945,7.551514e-29,0.0,3.475304e-37,1.0,2.015067e-42,3.0,65909173.0,-16.118096


In [3]:
result_dir = "/groups/icecube/cjb924/workspace/work/high_energy/5_model/predicting/predictions/no_1/signal_5model_prediction_burnsample_with_extra.csv"
result = pd.read_csv(result_dir).sort_values('event_no').reset_index(drop=True)
result
    
logit_computations_LE_nu = 'signal_neutrino_LE_pred'
logit_computations_HE_nu = 'signal_neutrino_HE_pred'

,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,signal_noise_pred,signal_neutrino_LE_pred,signal_neutrino_HE_pred,signal_muon_LE_pred,signal_muon_HE_pred,event_no,total_osc_weight,signal_neutrino_HE_pred_logit,L3_oscNext_bool,signal_neutrino_LE_pred_logit,signal_muon_LE_pred_logit,signal_muon_HE_pred_logit,signal_noise_pred_logit
0,0,0,0,0,0,0,7.013079e-01,2.907014e-01,0.006712,6.226310e-07,0.001278,0.0,2.553499e-07,-4.997049,0.0,-0.891980,-14.140367,-6.661331,0.853534
1,1,1,1,1,1,1,1.964039e-03,1.321530e-02,0.059956,3.540448e-08,0.924865,1.0,2.553499e-07,-2.752312,0.0,-4.313069,-15.814999,2.510353,-6.230735
2,2,2,2,2,2,2,2.244813e-01,5.933423e-01,0.091406,1.199663e-07,0.090770,2.0,2.553499e-07,-2.296586,0.0,0.377800,-15.329791,-2.304265,-1.239739
3,3,3,3,3,3,3,5.102842e-01,4.660947e-01,0.021795,9.134259e-07,0.001825,3.0,2.553499e-07,-3.804017,0.0,-0.135829,-13.802173,-6.304484,0.041143
4,4,4,4,4,4,4,8.564743e-01,1.286826e-01,0.013312,3.187059e-06,0.001528,4.0,2.553499e-07,-4.305663,0.0,-1.912657,-12.625514,-6.482460,1.786310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61739857,61739857,61739857,61739857,61739857,61739857,61739857,9.995605e-01,2.690546e-04,0.000058,9.025324e-11,0.000112,99904418.0,2.553499e-07,-9.754409,0.0,-8.219956,-16.117193,-9.091689,7.729285
61739858,61739858,61739858,61739858,61739858,61739858,61739858,7.906637e-02,7.302544e-01,0.029228,5.539359e-08,0.161451,99904419.0,2.553499e-07,-3.502947,1.0,0.995913,-15.677305,-1.647473,-2.455099
61739859,61739859,61739859,61739859,61739859,61739859,61739859,3.444787e-03,5.550002e-01,0.151944,1.231606e-07,0.289611,99904420.0,2.553499e-07,-1.719433,0.0,0.220895,-15.315374,-0.897275,-5.667414
61739860,61739860,61739860,61739860,61739860,61739860,61739860,5.715953e-08,1.219413e-04,0.013442,3.040365e-13,0.986436,99904421.0,2.553499e-07,-4.295832,0.0,-9.011029,-16.118093,4.286677,-15.666004


In [9]:
logit_computations_LE_nu = 'signal_neutrino_LE_pred'
logit_computations_HE_nu = 'signal_neutrino_HE_pred'
result_LE = result[logit_computations_LE_nu]
result_HE = result[logit_computations_HE_nu]
result_i = result_LE + result_HE
result_i

0           0.297414
1           0.073171
2           0.684748
3           0.487890
4           0.141995
              ...   
61739857    0.000327
61739858    0.759483
61739859    0.706944
61739860    0.013564
61739861    0.000004
Length: 61739862, dtype: float64

In [10]:
0.006712+2.907014e-01

0.2974134

In [2]:
connection = sqlite3.connect('/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_0.01_percent_burnsample_Peter/merged_db/Burnsample_lvl3_v02.00_lvl3_variables.db')

columns_select = ['EventID', 'SubEventID', 'RunID', 'SubrunID', 'L3_oscNext_bool', 'L4_oscNext_bool', 'L5_oscNext_bool', 'L6_oscNext_bool', 'L7_oscNext_bool']
query = f"SELECT {', '.join(columns_select)} FROM truth"
df_info = pd.read_sql_query(query, connection)

connection.close()

In [7]:
print('lvl 3:', df_info['L3_oscNext_bool'].unique())
print('lvl 4:', df_info['L4_oscNext_bool'].unique())
print('lvl 5:', df_info['L5_oscNext_bool'].unique())
print('lvl 6:', df_info['L6_oscNext_bool'].unique())
print('lvl 7:', df_info['L7_oscNext_bool'].unique())

lvl 3: [0. 1.]
lvl 4: [-1.]
lvl 5: [-1.]
lvl 6: [-1.]
lvl 7: [-1.]
